In [32]:
import numpy as np
import pandas as pd
import os
import random 
import csv
import zipfile
import shutil
import cv2
import pydicom as dicom

In [2]:
covid_chestxray_path = './covid-chestxray-dataset-master/metadata.csv'
kaggle_classinfo_path = './rsna-pneumonia-detection-challenge/stage_2_detailed_class_info.csv' 
kaggle_trainlabes_path = './rsna-pneumonia-detection-challenge/stage_2_train_labels.csv'
kaggle_images_path = './rsna-pneumonia-detection-challenge/stage_2_train_images'

In [3]:

covid_csv = pd.read_csv(covid_chestxray_path)
normal_csv = pd.read_csv(kaggle_classinfo_path)
pnemonia_csv = pd.read_csv(kaggle_trainlabes_path)

In [4]:
covid_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patientid               950 non-null    object 
 1   offset                  697 non-null    float64
 2   sex                     870 non-null    object 
 3   age                     713 non-null    float64
 4   finding                 950 non-null    object 
 5   RT_PCR_positive         593 non-null    object 
 6   survival                361 non-null    object 
 7   intubated               248 non-null    object 
 8   intubation_present      250 non-null    object 
 9   went_icu                397 non-null    object 
 10  in_icu                  335 non-null    object 
 11  needed_supplemental_O2  90 non-null     object 
 12  extubated               37 non-null     object 
 13  temperature             78 non-null     float64
 14  pO2_saturation          119 non-null    fl

In [5]:
covid_csv.head()

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,22-Jan-20,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,25-Jan-20,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,27-Jan-20,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,28-Jan-20,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,25-Jan-20,"Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [6]:
pnemonia_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30227 entries, 0 to 30226
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   patientId  30227 non-null  object 
 1   x          9555 non-null   float64
 2   y          9555 non-null   float64
 3   width      9555 non-null   float64
 4   height     9555 non-null   float64
 5   Target     30227 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 1.4+ MB


In [7]:
pnemonia_csv.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [8]:
pnemonia_csv.Target.unique()

array([0, 1])

In [10]:

related_views=["AP","PA","AP Supine","AP semi erect"]
image_labels = {'normal': [], 'pneumonia': [], 'COVID-19': []}
covid_file_labels = {'normal': [], 'pneumonia': [], 'COVID-19': []}
patients_id={'normal': [], 'pneumonia': [], 'COVID-19': []}
mapping = dict() 
mapping['Pneumonia/Viral/COVID-19'] = 'COVID-19'
mapping['Pneumonia/Fungal/Pneumocystis'] = 'pneumonia'
mapping['Pneumonia/Viral/SARS'] = 'pneumonia'
mapping['Pneumonia/Bacterial/Streptococcus'] = 'pneumonia'
mapping['No Finding'] = 'normal'
for index, row in covid_csv.iterrows():
    if row['finding'] in mapping and row['view'] in related_views and row['filename'] not in image_labels[mapping[row['finding']]]:
        image_labels[mapping[row['finding']]].append(row['filename'])
        covid_file_labels[mapping[row['finding']]].append(row['filename'])
        patients_id[mapping[row['finding']]].append(row['patientid'])

In [11]:
print('Normal Image Count   = ',len(image_labels['normal']))
print('Pnemonia Image Count = ',len(image_labels['pneumonia']))
print('Covid-19 Image Count = ',len(image_labels['COVID-19']))

Normal Image Count   =  18
Pnemonia Image Count =  58
Covid-19 Image Count =  478


In [12]:
normal_csv.head()

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity


In [13]:

patient_list=[]
for index, row in normal_csv.iterrows():
    if row['class'] == 'Normal' and row['patientId'] not in patient_list :
        patient_list.append(row['patientId'])
        imageinfo=row['patientId']+'.dcm'
        if imageinfo not in image_labels['normal']:
            image_labels['normal'].append(imageinfo)


In [14]:

for index, row in pnemonia_csv.iterrows():
    if int(row['Target']) == 1 and row['patientId'] not in patient_list:
        patient_list.append(row['patientId'])
        imageinfo=row['patientId']+'.dcm'
        if imageinfo not in image_labels['pneumonia']:
            image_labels['pneumonia'].append(imageinfo)

In [15]:
print('Normal Image Count   = ',len(image_labels['normal']))
print('Pnemonia Image Count = ',len(image_labels['pneumonia']))
print('Covid-19 Image Count = ',len(image_labels['COVID-19']))

Normal Image Count   =  8869
Pnemonia Image Count =  6070
Covid-19 Image Count =  478


In [ ]:
with open('All.csv',newline='', mode='w') as csvfile:
     csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
     csv_writer.writerow(['filename','class'])
     for key in image_labels:
         for row in image_labels[key]:
             if '.dcm' in row:
                 new_row=row[:-4]+'.png'
                 csv_writer.writerow([new_row,key])
             else:
                 csv_writer.writerow([row,key])     


In [16]:
#selecting random 45 images from covid data to filter out our test images
covid_test_data  = ['2','4','6','11','12','112','113','114','115','116','200','201','202','203','204','302','303','304','305','306','422','445','446','447','448','459']

In [ ]:
try:
  os.mkdir('Train_data')
except:
  pass

In [23]:
test_pnemonia_ids = ['8', '31','171']
test_file_generation = []
for index, row in covid_csv.iterrows(): 
    if  str(row['patientid']) in test_pnemonia_ids or str(row['patientid']) in covid_test_data:
        if row['filename'] not in test_file_generation:
            test_file_generation.append(row['filename'])
covid_train_data ={'normal': [], 'pneumonia': [], 'COVID-19': []}

for file_name in covid_file_labels['pneumonia']:
    if file_name not in test_file_generation:
        covid_train_data['pneumonia'].append(file_name)

for file_name in covid_file_labels['COVID-19']:
    if file_name not in test_file_generation:
        covid_train_data['COVID-19'].append(file_name)
print(len(covid_train_data['normal']),len(covid_train_data['pneumonia']),len(covid_train_data['COVID-19']))
print(len(image_labels['normal']),len(image_labels['pneumonia']),len(image_labels['COVID-19']))



0 50 433
8869 6070 478


In [31]:
temp_all_train={'normal': [], 'pneumonia': [], 'COVID-19': []}
for key in image_labels:
    for file_name in image_labels[key]:
        if file_name not in test_file_generation:
            if file_name not in covid_train_data[key]:
                temp_all_train[key].append(file_name) 
print(len(temp_all_train['normal']),len(temp_all_train['pneumonia']),len(temp_all_train['COVID-19']))

for i in range(10):
    for key in temp_all_train:
        random.shuffle(temp_all_train[key]) 
for i in range(1,9): 
    globals()['train{}'.format(i)]={'normal': [], 'pneumonia': covid_train_data['pneumonia'].copy(), 'COVID-19': covid_train_data['COVID-19'].copy()}
    globals()['train{}'.format(i)]['normal'].extend(temp_all_train['normal'][500*i:(500*i)+500])
    globals()['train{}'.format(i)]['pneumonia'].extend(temp_all_train['pneumonia'][450*i:(450*i)+450])
# print(len(train1['normal']),len(train1['pneumonia']),len(train1['COVID-19']))

for i in range(1,9): #Export CSV
    with open('Train_data/train{}.csv'.format(i),newline='', mode='w') as csvfile:
         all_rows=[]
         csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
         csv_writer.writerow(['filename','class'])
         for key in globals()['train{}'.format(i)]:
             for row in globals()['train{}'.format(i)][key]:
                 if '.dcm' in row:
                     new_row=row[:-4]+'.png'
                     all_rows.append([new_row,key])
                 else:
                     if '.png ' in row:
                         all_rows.append([row[:-1],key])
                     else:
                        all_rows.append([row,key])
         for j in range(10):
            random.shuffle(all_rows)
         for arow in all_rows:
            csv_writer.writerow(arow)
    print({'normal':len(globals()['train{}'.format(i)]['normal']),
           'pneumonia':len(globals()['train{}'.format(i)]['pneumonia']),
           'COVID-19':len(globals()['train{}'.format(i)]['COVID-19'])})



8869 6012 0
500 500 433
{'normal': 500, 'pneumonia': 500, 'COVID-19': 433}
{'normal': 500, 'pneumonia': 500, 'COVID-19': 433}
{'normal': 500, 'pneumonia': 500, 'COVID-19': 433}
{'normal': 500, 'pneumonia': 500, 'COVID-19': 433}
{'normal': 500, 'pneumonia': 500, 'COVID-19': 433}
{'normal': 500, 'pneumonia': 500, 'COVID-19': 433}
{'normal': 500, 'pneumonia': 500, 'COVID-19': 433}
{'normal': 500, 'pneumonia': 500, 'COVID-19': 433}


In [ ]:
#code to convert images into png format
All_data=pd.read_csv('All_data/All.csv', nrows=None) 
for index, row in All_data.iterrows(): 
  if '.png' in row['filename']:
    try:
      png_index=row['filename'].find('.png')
      last_name=row['filename'][:png_index]+'.dcm'
      ds = dicom.dcmread(os.path.join('rsna-pneumonia-detection-challenge/stage_2_train_images',last_name))
      pixel_array_numpy = ds.pixel_array
      imgname = last_name[:-4]+'.png'
      cv2.imwrite(os.path.join('All_data/All', imgname), pixel_array_numpy)
    except:
      png_index=row['filename'].find('.png')
      img=cv2.imread(os.path.join('covid-chestxray-dataset-master/images',row['filename'][:png_index+4])) 
      gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
      cv2.imwrite(os.path.join('All_data/All', row['filename'][:png_index+4]), gray)  
  else:
    img=cv2.imread(os.path.join('covid-chestxray-dataset-master/images',row['filename']))
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    cv2.imwrite(os.path.join('All_data/All', row['filename']), gray)  
    

In [ ]:
#code to generate all_test.csv
All=[] 
all_train=[] 
all_test=[]
with open('All/All.csv',newline='', mode='r') as csvfile: 
      csvreader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      for row in csvreader:
          All.append(row)
for i in range(1,9): 
  with open('Train_data/train{}.csv'.format(i),newline='', mode='r') as csvfile:
      csvreader = csv.reader(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      for row in csvreader:
        all_train.append(row)
with open('all_test.csv'.format(i),newline='', mode='w') as csvfile: 
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['filename','class'])
    for row in All:
      if row not in all_train:
        csvwriter.writerow(row)

In [ ]:
#code to generate val_data.csv
for i in range(10): 
  random.shuffle(All)
with open('val_data.csv'.format(i),newline='', mode='w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['filename','class'])
    normal_length=0
    pnemonia_length=0
    for row in All:
      if row not in all_train:
        if row[1]=='COVID-19':
          csvwriter.writerow(row)
        elif row[1]=='normal':
          if normal_length<400:
            csvwriter.writerow(row)
            normal_length+=1
        else:
          if pnemonia_length<400:
            csvwriter.writerow(row)
            pnemonia_length+=1